In [2]:
#AutoTraceDraw.py
import turtle as t

In [7]:
t.title('自动轨迹绘制') #窗口名称
t.setup(800, 600, 0, 0) #窗口尺寸
t.pencolor("red")
t.pensize(5)
#数据读取
datals = []
f = open("../Logs/TLog.txt")    #打开文件f，并附句柄为f
for line in f:    #逐行遍历文件f
    line = line.replace("\n","")    #去掉行尾空格
    datals.append(list(map(eval, line.split(",")))) #map函数，用参数一的函数遍历参数2的每个元素；这里line.split后是列表，元素是数字字符串
f.close()
#自动绘制
for i in range(len(datals)):
    t.pencolor(datals[i][3],datals[i][4],datals[i][5])  #脚本数据格式为：前进像素，左右转，转角度，rgb值1，rgb值2，rgb值3
    t.fd(datals[i][0])     #fd方法：前进像素
    if datals[i][1]:        #1右转0左转，
        t.rt(datals[i][2])    #rt方法：右转角度
    else:
        t.lt(datals[i][2])    #lt方法：左转角度


SyntaxError: invalid character in identifier (<ipython-input-7-fb8ab16b9af2>, line 7)

In [8]:
# AutoTrace.py
import turtle as t
t.title("自动轨迹绘制")
t.setup(800,600)
t.pencolor("red")
t.pensize(5)
t.speed(10)
# 数据读取
datals=[]
f=open("data.trace",'rt')
for line in f:
    line=line.replace('\n','')
    datals.append(list(map(eval,line.split(','))))
f.close()
# 自动绘制
for i in range(len(datals)):
    t.pencolor(datals[i][3],datals[i][4],datals[i][5])
    t.fd(datals[i][0])
    t.left(datals[i][2]) if datals[i][1]==0 else t.right(datals[i][2])
t.done()



FileNotFoundError: [Errno 2] No such file or directory: 'data.trace'

In [1]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cbook
from matplotlib import cm
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal
gdal.AllRegister()


ImportError: No module named 'osgeo'

In [ ]:
filePath = u"E:\桌面\dem实验/Himalaya.tif"  #输入你的dem数据

dataset = gdal.Open(filePath)
adfGeoTransform = dataset.GetGeoTransform()
band = dataset.GetRasterBand(1)   #用gdal去读写你的数据，当然dem只有一个波段

nrows  =  dataset.RasterXSize  
ncols   =  dataset.RasterYSize   #这两个行就是读取数据的行列数


Xmin = adfGeoTransform[0]  #你的数据的平面四至
Ymin = adfGeoTransform[3]
Xmax = adfGeoTransform[0] + nrows * adfGeoTransform[1] + ncols * adfGeoTransform[2]
Ymax = adfGeoTransform[3] + nrows * adfGeoTransform[4] + ncols * adfGeoTransform[5]

x = np.linspace(Xmin,Xmax, ncols)
y = np.linspace(Ymin,Ymax, nrows)
X,Y = np.meshgrid(x, y)
Z = band.ReadAsArray(0, 0,nrows, ncols) #这一段就是讲数据的x，y，z化作numpy矩阵

region = np.s_[10:400,10:400]
X, Y, Z = X[region], Y[region],Z[region]

fig, ax = plt.subplots(subplot_kw=dict(projection='3d'), figsize=(12,10))
ls = LightSource(270, 20)   #设置你可视化数据的色带
rgb = ls.shade(Z, cmap=cm.gist_earth, vert_exag=0.1, blend_mode='soft')
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, facecolors=rgb,
                        linewidth=0, antialiased=False, shade=False)

plt.show()  #最后渲染出你好看的三维图吧


--------------------- 
作者：大虾卢 
来源：CSDN 
原文：https://blog.csdn.net/allenlu2008/article/details/51880333 
版权声明：本文为博主原创文章，转载请附上博文链接！

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as crs
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from PIL import Image
 
lat = 116
lon = 39
img = Image.open('Flag_of_China.png')
 
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=crs.PlateCarree())
ax.coastlines()
ax.stock_img()
imagebox = OffsetImage(img, zoom=0.01)
imagebox.image.axes = ax
ab = AnnotationBbox(imagebox, [lat, lon], pad=0, frameon=False)
ax.add_artist(ab)
 
plt.show()

Invalid argument (must be a Polygon)


In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
    
from netCDF4 import Dataset   
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from mpl_toolkits.basemap import Basemap

from wrf import to_np, getvar, smooth2d, get_basemap, latlon_coords

# Open the NetCDF file
ncfile = Dataset("/Users/ladwig/Documents/wrf_files/wrfout_d01_2016-10-07_00_00_00")

# Get the sea level pressure
slp = getvar(ncfile, "slp")

# Smooth the sea level pressure since it tends to be noisy near the mountains
smooth_slp = smooth2d(slp, 3)

# Get the latitude and longitude points
lats, lons = latlon_coords(slp)

# Get the basemap object
bm = get_basemap(slp)

# Create a figure
fig = plt.figure(figsize=(12,9))

# Add geographic outlines
bm.drawcoastlines(linewidth=0.25)
bm.drawstates(linewidth=0.25)
bm.drawcountries(linewidth=0.25)

# Convert the lats and lons to x and y.  Make sure you convert the lats and lons to 
# numpy arrays via to_np, or basemap crashes with an undefined RuntimeError.
x, y = bm(to_np(lons), to_np(lats))

# Draw the contours and filled contours
bm.contour(x, y, to_np(smooth_slp), 10, colors="black")
bm.contourf(x, y, to_np(smooth_slp), 10, cmap=get_cmap("jet"))

# Add a color bar
plt.colorbar(shrink=.62)

plt.title("Sea Level Pressure (hPa)")

plt.savefig("/Users/ladwig/Documents/workspace/wrf_python/doc/source/_static/images/basemap_slp.png",
           transparent=True, bbox_inches="tight")

plt.show()

ImportError: cannot import name 'is_scalar'

In [1]:
from __future__ import print_function
    
from netCDF4 import Dataset
from wrf import getvar, get_basemap, latlon_coords, geo_bounds

ncfile = Dataset("/Users/ladwig/Documents/wrf_files/wrfout_d01_2016-10-07_00_00_00")

slp = getvar(ncfile, "slp")

# Get the basemap mapping object
basemap_proj = get_basemap(slp)

print (basemap_proj)

# Get the latitude and longitude coordinate.  This is needed for plotting.
lats, lons = latlon_coords(slp)

# Get the geobounds for the full domain
bounds = geo_bounds(slp)

print(bounds)

# Get the geographic boundaries for a subset of the domain
slp_subset = slp[150:250, 150:250]
slp_subset_bounds = geo_bounds(slp_subset)

print (slp_subset_bounds)

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/ladwig/Documents/wrf_files/wrfout_d01_2016-10-07_00_00_00'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

ImportError: No module named 'cartopy'

In [1]:
plt.figure(figsize=(6, 3))
ax=plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
ax.coastlines(resolution='110m')
ax.gridlines()

NameError: name 'plt' is not defined

In [ ]:
#===================================================
#函数形式，调用cartopy，绘制全球地图
#===================================================
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def make_map(scale):
    fig=plt.figure(figsize=(8, 10))
    ax=plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
    ax.set_global()
    land = cfeature.NaturalEarthFeature('physical', 'land', scale,edgecolor='face',
                                                              facecolor=cfeature.COLORS['land'])
    ax.add_feature(land, facecolor='0.75')
    ax.coastlines(scale)
    #标注坐标轴
    ax.set_xticks([0, 60, 120, 180, 240, 300, 360], crs=ccrs.PlateCarree())
    ax.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=ccrs.PlateCarree())
    #zero_direction_label用来设置经度的0度加不加E和W
    lon_formatter = LongitudeFormatter(zero_direction_label=False)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    #添加网格线
    #gl = ax.gridlines()
    ax.grid()
    return fig,ax
fig,ax=make_map(scale='110m')


https://blog.csdn.net/u010159842/article/details/70738854

In [ ]:
import numpy as np
import cartopy.crs as crs
import cartopy.io.shapereader as shpreader

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import shapely.geometry as sgeom


from netCDF4 import Dataset
from matplotlib.cm import get_cmap
from cartopy.feature import NaturalEarthFeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from wrf import getvar, interplevel, to_np, latlon_coords, get_cartopy, cartopy_xlim, cartopy_ylim,  smooth2d

# Open the NetCDF file
ncfile = Dataset("wrfout_d01_2017-01-26_00_00_00")

# Extract the pressure, geopotential height, and wind variables
p = getvar(ncfile, "pressure")
z = getvar(ncfile, "z", units="dm")
ua = getvar(ncfile, "ua", units="kt")
va = getvar(ncfile, "va", units="kt")
wspd = getvar(ncfile, "wspd_wdir", units="kts")[0,:]

# Interpolate geopotential height, u, and v winds to 500 hPa
ht_500 = interplevel(z, p, 500)

ht_500 = smooth2d(ht_500,5)
u_500 = interplevel(ua, p, 500)
v_500 = interplevel(va, p, 500)
wspd_500 = interplevel(wspd, p, 500)

# Get the lat/lon coordinates
lats, lons = latlon_coords(ht_500)

# Get the map projection information
cart_proj = get_cartopy(ht_500)

# Create the figure
fig = plt.figure(figsize=(12,9))
ax = plt.axes(projection=cart_proj)

# Download and add the states and coastlines
china_border = r"/home/disk3/jiaobf/test-cartopy/bou2_4m/bou2_4p.shp"  #预先准备好的精准中国地图
adm1_shapes = list(shpreader.Reader(china_border).geometries())    #读取地图文件信息
ax.add_geometries(adm1_shapes, crs.PlateCarree(),edgecolor='black', facecolor='none', alpha=0.5)

###########################################################################################3
#参考 https://code.csdn.net/Kevin_QQ/cartopy_trial/tree/master
#lon=[125,126]   #比较笨的方法，通过落在省内的经纬度对，选中目标省份
#lat=[43,44]
#track = sgeom.LineString(zip(lon, lat))
#for state in shpreader.Reader(china_border).geometries():
#    # pick a default color for the land with a black outline,
#    # this will change if the storm intersects with our track
#    facecolor = [0.9375, 0.9375, 0.859375]
#    edgecolor = 'black'
#
#    if state.intersects(track):
#        facecolor = 'red'
#        ax.add_geometries([state], crs.PlateCarree(),facecolor='None', edgecolor='red')
#############################################################################################

#根据省份的名字，选择所需要的省份，注意中国边界文件中的编码方式为GBK
for state, geos in zip(shpreader.Reader(china_border).records(), shpreader.Reader(china_border).geometries()) :
#方法1
    if not state.attributes['NAME']=='':   #会存在一些空的，导致GBK解码异常
        statename = state.attributes['NAME'].decode('GBK')
       # print(statename)

    if statename == '黑龙江省' or \
       statename == '吉林省' or \
       statename == '辽宁省':
        ax.add_geometries([geos], crs.PlateCarree(),facecolor='None', edgecolor='red')

#方法二
#    if state.attributes['NAME'] == '黑龙江省'.encode('GBK') or \
#       state.attributes['NAME'] == '吉林省'.encode('GBK') or \
#       state.attributes['NAME'] == '辽宁省'.encode('GBK'):
#        ax.add_geometries([geos], crs.PlateCarree(),facecolor='None', edgecolor='red')



# Add the 500 hPa geopotential height contours
levels = np.arange(520., 580., 6.)
contours = plt.contour(to_np(lons), to_np(lats), to_np(ht_500), levels=levels, colors="black",
                       transform=crs.PlateCarree())
plt.clabel(contours, inline=1, fontsize=10, fmt="%i")   #为等值线图添加标签

# Add the wind speed contours
levels = [25, 30, 35, 40, 50, 60, 70, 80, 90, 100, 110, 120]
wspd_contours = plt.contourf(to_np(lons), to_np(lats), to_np(wspd_500), levels=levels,
                             cmap=get_cmap("rainbow"),
                             transform=crs.PlateCarree())
plt.colorbar(wspd_contours, ax=ax, orientation="horizontal", pad=.05, shrink=.62)

# Add the 500 hPa wind barbs, only plotting every 125th data point.
plt.barbs(to_np(lons[::125,::125]), to_np(lats[::125,::125]), to_np(u_500[::125, ::125]),
          to_np(v_500[::125, ::125]), transform=crs.PlateCarree(), length=6)

# Set the map bounds
ax.set_xlim(cartopy_xlim(ht_500))
ax.set_ylim(cartopy_ylim(ht_500))

g1 = ax.gridlines(linewidth=2, color='red', alpha=0.5, linestyle='--', crs = crs.PlateCarree())
#g1.xlines = False  
g1.xlocator = mticker.FixedLocator([105, 110, 115, 120, 125, 130, 135, 140, 145])  #指定网格线的经纬度
g1.ylocator = mticker.FixedLocator([30, 35, 40, 45, 50, 55])
#除了latlon投影和墨卡托投影，无法将经纬度标签放置在坐标轴上

plt.title("500 MB Height (dm), Wind Speed (kt), Barbs (kt)")

plt.show()



Invalid argument (must be a Polygon)
Geometry must be a Point or LineString


https://www.jianshu.com/p/b2bd14f63ece

In [3]:
from __future__ import print_function

from netCDF4 import Dataset
from wrf import getvar

ncfile = Dataset("wrfout_d01_2016-10-07_00_00_00")

# Get the Sea Level Pressure
slp = getvar(ncfile, "slp")

print(slp)

FileNotFoundError: [Errno 2] No such file or directory: b'wrfout_d01_2016-10-07_00_00_00'

In [ ]:
from __future__ import print_function

from netCDF4 import Dataset
from wrf import getvar

ncfile = Dataset("wrfout_d01_2016-10-07_00_00_00")

p = getvar(ncfile, "P")

print(p)

Invalid argument (must be a Polygon)


In [6]:
# urllib2_request.py
import urllib.request
import urllib

import json


def download_pois(keywords,types,city,children,offset,page,extensions,key):
    keywords = keywords
    types = types   
    city = city
    children = children
    offset = offset
    page = page
    extensions  = extensions
    key = key
    # keywords =  '北京大学'
    # types = '高等院校'
    # city = '北京'
    # children = 1
    # offset = 20
    # page = 1
    # extensions = 'all'
    # key ='b9ca5e60b99fda0fc6fac6644f88c814'
    url = 'http://restapi.amap.com/v3/place/text?key='+str(key)+'&keywords='+str(keywords)+'&types='+str(types)+'&city='+str(city)+'&children='+str(children)+'&offset='+str(offset)+'&page'+str(page)+'&extensions='+str(extensions)
    response = urllib.request.urlopen(url)
    res = response.read()
    file = open('1.txt','wb')
    file.write(res)
    file.close()



if __name__ == '__main__':
    download_pois('北京大学','高等院校','北京','1','20','1','all','b9ca5e60b99fda0fc6fac6644f88c814')


UnicodeEncodeError: 'ascii' codec can't encode characters in position 65-68: ordinal not in range(128)

--------------------- 
作者：staHuri 
来源：CSDN 
原文：https://blog.csdn.net/staHuri/article/details/77851373 
版权声明：本文为博主原创文章，转载请附上博文链接！

In [7]:
import urllib.request
resp=urllib.request.urlopen("http://www.baidu.com")